In [ ]:
serie_test = data_orbf[(data_orbf.indicator_label == "Nombre de nouvelles acceptances pour les méthodes contraceptives de longue durée(DIU et implant)") & (data_orbf.entity_id == 3)]
#autocorrelation_plot(serie_test.indicator_verified_value)
serie_test = serie_test.set_index('date')

In [3]:
from __future__ import print_function
import os.path
from itertools import chain
from jupyter_core.paths import jupyter_data_dir
from notebook.nbextensions import _get_nbextension_dir as get_nbext_dir

nbextensions_dirs = (get_nbext_dir() , os.path.join(jupyter_data_dir() , 'nbextensions'))

In [ ]:
yaml_files = []
for root , dirs ,  files in chain.from_iterable(os.walk(nb_ext_dir , followlinks=True) for nb_ext_dir in nb_ext_di)

In [ ]:
plt.plot(serie_test.indicator_verified_value)

In [ ]:
# fit model
model = ARIMA(serie_test.indicator_verified_value, order=(1,0,0))
model_fit = model.fit(disp=0)
#print(model_fit.summary())
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
residuals.plot(legend = False)
plt.show()
residuals.plot(kind='kde' , legend = False)
plt.show()
#print(residuals.describe())

In [ ]:
X_obs = serie_test.indicator_claimed_value
X_val = serie_test.indicator_verified_value

size = int(len(X_obs) * 0.1)

train, test = X_val[0:size], X_obs[size:len(X_obs)]
verified = X_val[size : len(X_obs)]
history = [x for x in train]
predictions = []
vals = []
z_scores = []
alarms = []

for t in range(len(test)):
    model = ARIMA(history , order=(1,0,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0].tolist()[0]
    predictions.append(yhat)
    obs = test[t]
    z_score = np.abs((yhat - obs) / (model_fit.resid.std()))
    if  z_score < 2:   
        vals.append(obs)
        history.append(obs)
        alarms.append(False)
    if z_score >= 2 :
        vals.append(verified[t])
        history.append(verified[t])
        alarms.append(True)

    z_scores.append(z_score)
error = mean_squared_error(test, predictions)
z_score = np.abs((pd.Series(predictions) - pd.Series(test.tolist())) / (model_fit.resid.std()))
print('Test MSE: %.3f' % error)

<div class="cite2c-biblio"></div>